### RAA_Instagram기사_양식통일

    1. Web : https://www.instagram.com/

    2. 채널 : 
        a. CNBC
        b. 해럴드경제

    3. URL 특징
        a.채널 마다 variables=%7B%22id%22%3A% 이후 부분이 다름
        b.URL json에 다음페이지 번호있음 (js['page_info']['end_cursor'][:-2])
            - URL을 규칙적으로 알 수 있어서 굳이 selenium 안써도 될 것 같았으나 request모듈로는 request error 발생 
        c.date가 timestamp 형식으로 적혀있음
        d.while문 돌다가 로그인하라는 페이지 나올 수 있음 
            - 로그인하면 그뒤로는 계속 크롤링 가능
            
    4. 양식통일
        a.뉴스제목 : None
        b.게시일자
        c.뉴스매체구분코드
        d.뉴스본문내용
        e.뉴스URL : 인스타그램 페이지
        f.국문영문구분코드
        g.긍정용어수 : None
        h.부정용어수 : None
        i.긍정반응수 : 좋아요수
        j.부정반응수 : None


In [1]:
from selenium import webdriver
import json
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import time

In [5]:
ID = []
Like_count = []
Comment_count = []
Main = []
Date = []

driver = webdriver.Chrome("C:\\Users\\shic\\chromedriver_win32\\chromedriver.exe")
url = "https://www.instagram.com/graphql/query/?query_hash=8c2a529969ee035a5063f2fc8602a0fd&variables=%7B%22id%22%3A%22259533357%22%2C%22first%22%3A55%2C%22after%22%3A%22QVFBdkNydzloNmJTUndjamJaVmV4QVBTVmJtNkFoTHZ4ZklwOVR2U0NJaDlRcE9hanQ4TEFLUXlCN2xmaDFLaU5Fc0JIZklFNUJhdU1YV3ZfeS1GRVdnQQ%3D%3D%22%7D"
url_format = "https://www.instagram.com/graphql/query/?query_hash=8c2a529969ee035a5063f2fc8602a0fd&variables=%7B%22id%22%3A%22259533357%22%2C%22first%22%3A55%2C%22after%22%3A%22{}%3D%3D%22%7D"

target_time = time.mktime(datetime.datetime.strptime("2010-01-01",'%Y-%m-%d').timetuple())

In [6]:
while 1:
    driver.get(url)
    try :
        if json.loads(BeautifulSoup(driver.page_source,"html.parser").text)['message'] == "rate limited":
            time.sleep(10)
            continue
    except:    
        js = json.loads(BeautifulSoup(driver.page_source,"html.parser").text)
        if 'data' not in js :
            break
        else :
            js = js['data']['user']['edge_owner_to_timeline_media']
            for num in range(len(js['edges'])):
                if len(js['edges'][num]['node']['edge_media_to_caption']['edges']) == 0:
                    continue
                else:
                    ID.append(js['edges'][num]['node']['shortcode']) # ID
                    Like_count.append(js['edges'][num]['node']['edge_media_preview_like']['count']) # 좋아요 수
                    Comment_count.append(js['edges'][num]['node']['edge_media_to_comment']['count']) # 댓글 수
                    mm = " ".join([js['edges'][num]['node']['edge_media_to_caption']['edges'][tmp]['node']['text'] for tmp in range(len(js['edges'][num]['node']['edge_media_to_caption']['edges']))]).replace("\u2060","").replace("\n","")
                    Main.append(mm) # 본문
                    Date.append(datetime.datetime.fromtimestamp(int(js['edges'][num]['node']['taken_at_timestamp'])).strftime('%Y-%m-%d %H:%M:%S')) # Date
            url = url_format.format(js['page_info']['end_cursor'][:-2])
            if int(js['edges'][num]['node']['taken_at_timestamp']) < target_time:
                break
            else:
                time.sleep(1)
    

In [73]:
CNBC = pd.DataFrame({"ID":ID,
                     "Like_count":Like_count,
                     "Comment_count":Comment_count,
                     "Main":Main,
                     "Date":Date,
                     "Chanel":"CNBC"})

In [76]:
CNBC = CNBC.drop_duplicates()

In [77]:
CNBC.to_csv("C:/Users/shic/Desktop/DH/price/CNBC.csv")

In [78]:
CNBC

,ID,Like_count,Comment_count,Main,Date,Chanel
0,CReawu6gy4t,7918,246,Sometimes it takes a major life event to push ...,2021-07-19 01:30:28,CNBC
1,CReC0XfJW4g,5993,209,The Amazon rainforest is emitting more carbon ...,2021-07-18 22:01:54,CNBC
2,CRc6m8hL2Kf,5072,162,China’s share of global bitcoin mining plunged...,2021-07-18 11:30:14,CNBC
3,CRcYStpD4Ue,8866,220,Facebook CEO Mark Zuckerberg this week announc...,2021-07-18 06:30:19,CNBC
4,CRcATTNLQBF,6472,101,The ability to create strong relationships is ...,2021-07-18 03:01:37,CNBC
...,...,...,...,...,...,...
10158,SxlCcutHle,14,0,Pizza for breakfast on @squawkcnbc. Don't mind...,2012-12-03 21:38:33,CNBC
10159,SqXZ2CNHlb,7,0,Markets reporter @bobpisani wrapping up a segm...,2012-12-01 02:24:43,CNBC
10160,SqRw-qNHgt,10,0,"Hey, it's @carlquintanilla LIVE from #post9.",2012-12-01 01:35:27,CNBC
10161,SqO9PKtHu0,16,0,It's the last trading day in November. We're t...,2012-12-01 01:10:55,CNBC


In [15]:
url = "https://www.instagram.com/graphql/query/?query_hash=8c2a529969ee035a5063f2fc8602a0fd&variables=%7B%22id%22%3A%222955659478%22%2C%22first%22%3A55"
after = ""
end = "%7D"

ID = []
Like_count = []
Comment_count = []
Main = []
Date = []

driver = webdriver.Chrome("C:\\Users\\shic\\chromedriver_win32\\chromedriver.exe")

while 1:
    driver.get(url + after + end)
    try:    
        js = json.loads(BeautifulSoup(driver.page_source,"html.parser").text)
        if 'data' not in js :
            break
        else: 
            js = js['data']['user']['edge_owner_to_timeline_media']
            for num in range(len(js['edges'])):
                if len(js['edges'][num]['node']['edge_media_to_caption']['edges']) == 0:
                    continue
                else:
                    ID.append(js['edges'][num]['node']['shortcode']) # ID
                    Like_count.append(js['edges'][num]['node']['edge_media_preview_like']['count']) # 좋아요 수
                    Comment_count.append(js['edges'][num]['node']['edge_media_to_comment']['count']) # 댓글 수
                    mm = " ".join([js['edges'][num]['node']['edge_media_to_caption']['edges'][tmp]['node']['text'] for tmp in range(len(js['edges'][num]['node']['edge_media_to_caption']['edges']))]).replace("\u2060","").replace("\n","")
                    Main.append(mm) # 본문
                    Date.append(datetime.datetime.fromtimestamp(int(js['edges'][num]['node']['taken_at_timestamp'])).strftime('%Y-%m-%d %H:%M:%S')) # Date
            after = "%2C%22after%22%3A%22{}%3D%3D%22".format(js['page_info']['end_cursor'][:-2])
    #        url = url_format.format(js['page_info']['end_cursor'][:-2])
            if int(js['edges'][num]['node']['taken_at_timestamp']) < target_time:
                break
            else:
                    time.sleep(1)
    except:
        time.sleep(10)


In [108]:
herald = pd.DataFrame({"ID":ID,
                     "Like_count":Like_count,
                     "Comment_count":Comment_count,
                     "Main":Main,
                     "Date":Date,
                     "Chanel":"Herald"})

In [114]:
instagram = pd.concat([CNBC,herald],
                      ignore_index=True)

In [115]:
# instagram.to_csv("C:/Users/shic/Desktop/DH/price/instagram.csv")

In [ ]:
instagram = instagram.rename(columns={
    'Like_count' : '좋아요건수',
    'Comment_count' : '댓글수',
    'Main' : '뉴스본문내용',
    'Date' : '기준일자',
    'ID' : '뉴스URL',
    'Chanel' : '외부뉴스구분코드'
})

instagram['뉴스URL'] = instagram.뉴스URL.map(lambda x: "https://www.instagram.com/p/" + x)
instagram['기준일자'] = instagram.기준일자.map(lambda x : x.split(" ")[0].replace("-",""))
instagram['외부뉴스구분코드'] = instagram.외부뉴스구분코드.map(lambda x: 'C' if x == 'CNBC' else 'D')
instagram = instagram.loc[:,["기준일자","외부뉴스구분코드","뉴스URL","뉴스본문내용","좋아요건수","댓글수"]]
instagram

In [19]:
instagram = instagram.rename(columns={
    '기준일자' : '게시일자',
    '외부뉴스구분코드' : '뉴스매체구분코드',
    '좋아요건수' : '긍정반응수'
})

instagram['뉴스제목'] = None
instagram['긍정용어수'] = None
instagram['부정용어수'] = None
instagram['부정반응수'] = None
instagram['국문영문구분코드'] = df.뉴스매체구분코드.map(lambda x: '1' if x == 'C' else "0" )


In [20]:
instagram = instagram.loc[:,['뉴스제목','게시일자','뉴스매체구분코드','뉴스본문내용','뉴스URL','국문영문구분코드','긍정용어수','부정용어수','긍정반응수','부정반응수']]

In [22]:
instagram.to_csv('RAA_Instagram기사_뉴스칼럼명통일.csv',
         index = False,encoding="utf-8-sig")